In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from torch.utils.data import DataLoader, TensorDataset, Dataset
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import matplotlib.dates as mdates

In [20]:
df = pd.read_csv('ColoradoData_cleaned.csv')
df = df.loc[df['Station_Name'] == 'BOULDER / JUNCTION ST1']
df.index = df['Start_DateTime']
df = df[['Start_DateTime', 'Energy_Consumption']].sort_index()
df.dropna(inplace=True)
df['Start_DateTime'] = pd.to_datetime(df['Start_DateTime'], format='%Y-%m-%d %H:%M:%S')
df.set_index('Start_DateTime', inplace=True)
print(df)

                     Energy_Consumption
Start_DateTime                         
2018-01-01 17:49:00               6.504
2018-01-01 17:49:00               6.504
2018-01-02 08:52:00               2.481
2018-01-02 08:52:00               2.481
2018-01-02 21:11:00              15.046
...                                 ...
2019-08-06 08:48:00               0.000
2019-08-07 08:56:00               0.000
2019-08-07 08:56:00               0.000
2019-08-07 10:17:00              11.060
2019-08-07 10:17:00              11.060

[968 rows x 1 columns]


In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

all_data = df['Energy_Consumption'].values.astype(float)
train_size = int(0.6 * len(df))
train_set = all_data[:train_size]
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_set.reshape(-1, 1))
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

## Define Model

In [21]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size), torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

## Define Objective Function

In [25]:
def objective(trial):
    # Hyperparameters to tune
    hidden_size = trial.suggest_int('hidden_size', 128, 512)
    learning_rate = trial.suggest_float('lr', 1e-4, 1e-1, log=True)
    train_window = trial.suggest_int('train_window', 12, 160)
    epochs = 150

    train_inout_seq = create_inout_sequences(train_data_normalized, train_window)
    model = LSTM()
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()

    for i in range(epochs):
        for seq, labels in train_inout_seq:
            optimizer.zero_grad()
            model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size), torch.zeros(1, 1, model.hidden_layer_size))
            y_pred = model(seq)
            loss = loss_function(y_pred, labels)
            loss.backward()
            optimizer.step()
    return loss.item()

In [26]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print("Best Hyperparameters:", study.best_params)

[I 2025-03-12 11:14:02,128] A new study created in memory with name: no-name-a2244aa0-fba4-4a6e-9a0e-cbbb6e6fa43d
[I 2025-03-12 11:14:18,392] Trial 0 finished with value: 0.002392837544903159 and parameters: {'hidden_size': 146, 'lr': 0.0001068119340190827, 'train_window': 157}. Best is trial 0 with value: 0.002392837544903159.
[I 2025-03-12 11:14:28,915] Trial 1 finished with value: 0.0031860314775258303 and parameters: {'hidden_size': 146, 'lr': 0.00013237342996661773, 'train_window': 53}. Best is trial 0 with value: 0.002392837544903159.
[I 2025-03-12 11:14:40,660] Trial 2 finished with value: 0.0006472834502346814 and parameters: {'hidden_size': 185, 'lr': 0.045227265016856394, 'train_window': 82}. Best is trial 2 with value: 0.0006472834502346814.
[I 2025-03-12 11:14:54,039] Trial 3 finished with value: 0.0031922655180096626 and parameters: {'hidden_size': 190, 'lr': 0.00024447533579319356, 'train_window': 139}. Best is trial 2 with value: 0.0006472834502346814.
[I 2025-03-12 11:1

Best Hyperparameters: {'hidden_size': 185, 'lr': 0.045227265016856394, 'train_window': 82}
